In [7]:
import numpy as np
import tensorflow as tf
from ESPCN import *
from utils import *

import os
import scipy.misc
import numpy as np
from utils import *
import tensorflow as tf
from easydict import EasyDict as edict

%reload_ext autoreload
%autoreload 2

config = edict()

config.sample_dir = "samples"
config.checkpoint_dir = "checkpoint"
config.log_dir = "logs"
config.train_size = 100000000 # use large number if you have enough memory
config.valid_size = 10 # use large number if you have enough memory
config.test_size = 6400 # use large number if you have enough memory
config.batch_size = 8 # use large number if you have enough memory
config.patch_size = 17
config.patch_shape = [51,51,3] #[51,51,3]
config.scale = 3 #3
config.learning_rate = 1e-5
config.epoch = 1000000
#config.channels = 3
#config.mode = "RGB"

config.channels = 1
config.mode = "YCbCr"

#'''
config.dataset = "DIV2K"
config.valid = edict()
config.valid.hr_path = "/home/johnyi/deeplearning/research/SISR_Datasets/train/T91_patch"

config.test = edict()
config.test.hr_path = "/home/johnyi/deeplearning/research/SISR_Datasets/test/"

config.train = edict()
config.train.hr_path = "/home/johnyi/deeplearning/research/SISR_Datasets/train/"

#'''
config.train.lr_init = 1e-3
config.train.lr_decay = 0.5
config.train.decay_iter = 10
config.augmentation = True

if not os.path.exists(config.checkpoint_dir):
    os.makedirs(config.checkpoint_dir)
if not os.path.exists(config.sample_dir):
    os.makedirs(config.sample_dir)
if not os.path.exists(config.log_dir):
    os.makedirs(config.log_dir)

In [2]:
start_time = time.time()
imdb = create_imdb(config)

loading from.. /home/johnyi/deeplearning/research/SISR_Datasets/train/DIV2K num images: 800
800 images loaded! setting took: 66.9466s


In [8]:
tf.reset_default_graph()
imdb = []
sess = tf.Session()
espcn = ESPCN(sess, config, imdb)

Number of variables in network: 8 , full list: [<tf.Variable 'CONV_1/kernel:0' shape=(5, 5, 1, 64) dtype=float32_ref>, <tf.Variable 'CONV_1/bias:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'CONV_2/kernel:0' shape=(3, 3, 64, 32) dtype=float32_ref>, <tf.Variable 'CONV_2/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'CONV_3/kernel:0' shape=(3, 3, 32, 9) dtype=float32_ref>, <tf.Variable 'CONV_3/bias:0' shape=(9,) dtype=float32_ref>, <tf.Variable 'CONV_OUT/kernel:0' shape=(1, 1, 1, 1) dtype=float32_ref>, <tf.Variable 'CONV_OUT/bias:0' shape=(1,) dtype=float32_ref>]


In [9]:
with sess.as_default():
    start_time = time.time()
    a,b = espcn.test(name = "Set5", load = True)
    print(time.time()-start_time)
    print("avg:",a,"bicubic:",b)

 [*] Reading checkpoints...
loading from  checkpoint/DIV2K
INFO:tensorflow:Restoring parameters from checkpoint/DIV2K/ESPCN-YCbCr-x3
 [*] Load SUCCESS
[Set5] image 0 shape: (510, 510, 3) downsampled: (170, 170, 3) PSNR: 32.48903723772342 bicubic: 31.805646589863137
[Set5] image 1 shape: (288, 288, 3) downsampled: (96, 96, 3) PSNR: 31.91600862904757 bicubic: 30.325163251618076
[Set5] image 2 shape: (255, 255, 3) downsampled: (85, 85, 3) PSNR: 23.811765385841003 bicubic: 21.970475997093768
[Set5] image 3 shape: (279, 279, 3) downsampled: (93, 93, 3) PSNR: 31.982014964513155 bicubic: 31.175769560331602
[Set5] image 4 shape: (342, 228, 3) downsampled: (114, 76, 3) PSNR: 28.27546315052631 bicubic: 26.494231107799692
2.684704542160034
avg: 29.694857873530292 bicubic: 28.354257301341256


In [11]:
with sess.as_default():
    espcn.train(config, True)

 [*] Reading checkpoints...
loading from  checkpoint/DIV2K
INFO:tensorflow:Restoring parameters from checkpoint/DIV2K/ESPCN-YCbCr-x3
 [*] Load SUCCESS
Epoch: [ 0] [   1/ 100] time: 0.0908, loss: 154.94918823
[Set5] image 0 shape: (510, 510, 3) downsampled: (170, 170, 3) PSNR: 32.55960756146753 bicubic: 31.805646589863137
[Set5] image 1 shape: (288, 288, 3) downsampled: (96, 96, 3) PSNR: 31.418299957502075 bicubic: 30.325163251618076
[Set5] image 2 shape: (255, 255, 3) downsampled: (85, 85, 3) PSNR: 23.63787625422318 bicubic: 21.970475997093768
[Set5] image 3 shape: (279, 279, 3) downsampled: (93, 93, 3) PSNR: 31.978975409086054 bicubic: 31.175769560331602
[Set5] image 4 shape: (342, 228, 3) downsampled: (114, 76, 3) PSNR: 28.233191401160795 bicubic: 26.494231107799692
Epoch: [ 0] test PSNR: 29.565590, bicubic: 28.354257
Epoch: [100] [   1/ 100] time: 45.8638, loss: 61.60775757
Epoch: [200] [   1/ 100] time: 88.7345, loss: 135.09860229
Epoch: [300] [   1/ 100] time: 131.7393, loss: 92.4

KeyboardInterrupt: 